In [2]:
import json
import pandas as pd

from utils import prepare_input
from main_model import two_stage_model

import matplotlib.pyplot as plt

%matplotlib inline

In [79]:
n_scenario = 16
budget_vector = [0,20,40,60,80]
objective_value = {}
decisions_set_norta = {}

for scenario_id in range(15):
    print(scenario_id)
    
    objective_value[scenario_id] = {}
    decisions_set_norta[scenario_id] = {}
    
    dir_name = 'norta_sm_' + str(n_scenario) + "_" + str(scenario_id) +'/'
    path = "/work/07346/ashukla/ls6/Norta_Results/" + dir_name
    
    with open(path + "model_params.json") as file:
        model_params = json.load(file)
    
    model_params["path_to_output"] = "Error_Path"
    model_params["input1"], model_params["input2"] = prepare_input(model_params["path_to_input"])
    model_params["mip_gap"] = 0
    
    base_model = two_stage_model(model_params)        
    base_model.model.update()
    
    for budget in budget_vector:
                
        sol_path = path + str(budget) + "M_solution.sol"
        base_model.model.read(sol_path)
        base_model.model.update()
        
        starting_solutions  = {}
        for sub_id in base_model.unique_substations:
            temp = round(base_model.model.getVarByName('x[' +  str(sub_id) + ']').Start)
            starting_solutions[sub_id] = temp
        
        decisions_set_norta[scenario_id][budget] = starting_solutions
        base_model.budget_ref.rhs = budget*1e6    
        temp_constraint =  base_model.model.addConstrs((base_model.x[substation] == starting_solutions[substation]) 
                                                   for substation in base_model.unique_substations)
                    
        base_model.model.setParam("LogFile", model_params["path_to_output"] + str(budget) + "M")
        base_model.model.setParam("MIPGap", model_params["mip_gap"])
        base_model.model.setParam("TimeLimit", model_params["time_limit"])
        base_model.model.setParam("Method", model_params["solver_method"])
        base_model.model.optimize()
        objective_value[scenario_id][budget] = base_model.model.ObjVal/10
        base_model.model.remove(temp_constraint)
        print("\n\n\n\n")

0
Set parameter LogFile to value "Error_Path0M"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 32400
Set parameter Method to value 2
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: AMD EPYC 7763 64-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 212165 rows, 77908 columns and 590482 nonzeros
Model fingerprint: 0x91d3a014
Variable types: 55968 continuous, 21940 integer (21578 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+05]
  Objective range  [6e-02, 6e-02]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e-02, 2e+04]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R42845 by 0.057142857

Presolve removed 211284 rows and 76721 columns
Presolve time: 2.46s
Presolved: 881 rows, 1187 columns, 4022 nonzeros
Variable types: 1033 continuous, 154 integer (154 binary)
Found heuristic solut

   4   1.07554091e+01  5.97856319e+00  3.48e-11 3.55e-15  2.14e-03     3s
   5   9.66802535e+00  9.42370618e+00  1.26e-11 4.44e-16  1.09e-04     3s
   6   9.63945085e+00  9.63731545e+00  2.74e-11 2.22e-16  9.56e-07     3s
   7   9.63925234e+00  9.63925014e+00  5.58e-12 2.22e-16  9.82e-10     3s
   8   9.63925201e+00  9.63925201e+00  3.48e-12 2.22e-16  9.82e-16     3s

Barrier solved model in 8 iterations and 3.07 seconds (2.17 work units)
Optimal objective 9.63925201e+00


Root relaxation: objective 9.639252e+00, 77 iterations, 0.05 seconds (0.02 work units)
Another try with MIP start

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                       9.6392520    9.63925  0.00%     -    3s
     0     0    9.63925    0    1    9.63925    9.63925  0.00%     -    3s

Explored 1 nodes (77 simplex iterations) in 3.11 seconds (2.18 work units)
Thread count was 16 (of 16 ava

KeyboardInterrupt: 

In [10]:
objective_value = pd.read_csv("norta_solution_performance.csv", index_col="Unnamed: 0")
objective_value

,0,20,40,60,80
0,4.46206,1.662950,0.963925,0.570401,0.536268
1,4.46206,1.473335,0.540104,0.079875,0.019832
2,4.46206,1.387784,0.488334,0.064999,0.005728
3,4.46206,1.453220,0.575519,0.164224,0.094641
4,4.46206,1.529728,0.619717,0.224413,0.190291
5,4.46206,1.436235,0.545782,0.115373,0.072534
6,4.46206,1.414116,0.493746,0.070776,0.013336
7,4.46206,1.386993,0.490771,0.066087,0.008509
8,4.46206,1.491373,0.538668,0.103622,0.054002
9,4.46206,1.387908,0.502191,0.070941,0.014533


In [6]:
pd.DataFrame(target_objective_value).T

,0,20,40,60,80
0,4.368371,1.123990,0.344374,0.016249,5.684342e-14
1,4.369148,1.363024,0.475128,0.057802,5.684342e-14
2,4.686368,1.361298,0.470577,0.059658,5.684342e-14
3,4.390406,1.295576,0.441190,0.043805,5.684342e-14
4,4.469889,1.234017,0.391050,0.030253,5.684342e-14
5,4.580408,1.341821,0.444010,0.045446,5.684342e-14
6,4.246165,1.322382,0.473269,0.058119,5.684342e-14
7,4.486476,1.325988,0.457021,0.054014,5.684342e-14
8,4.253427,1.310243,0.455101,0.049416,5.684342e-14
9,4.500595,1.293950,0.461930,0.056169,5.684342e-14


# Estimated objective in Norta scenarios

In [8]:
n_scenario=16
budget_vector=[0,20,40,60,80]
target_objective_value = {}

def find_line_with_string(file_path, target_string):
    try:
        with open(file_path, 'r') as file:
            for line in file:
                if target_string in line:
                    return line
        return None  # If no line containing the target string is found
    except FileNotFoundError:
        print("File not found.")
        return None


for scenario_id in range(15):
    
    target_objective_value[scenario_id] = {}
    
    dir_name = 'norta_sm_' + str(n_scenario) + "_" + str(scenario_id) +'/'
    path = "/work/07346/ashukla/ls6/Norta_Results/" + dir_name
    
    for budget in budget_vector:
        file_path = path + str(budget) + "M"  # Replace with your file path
        target_string = "Best objective"
        found_line = find_line_with_string(file_path, target_string)
        target_objective_value[scenario_id][budget] = float(found_line.split(" ")[2][:-1])/10

In [9]:
pd.DataFrame(target_objective_value)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,4.368371e+00,4.369148e+00,4.686368e+00,4.390406e+00,4.469889e+00,4.580408e+00,4.246165e+00,4.486476e+00,4.253427e+00,4.500595e+00,4.550784e+00,4.691436e+00,4.428845e+00,4.513363e+00,4.507533e+00
20,1.123990e+00,1.363024e+00,1.361298e+00,1.295576e+00,1.234017e+00,1.341821e+00,1.322382e+00,1.325988e+00,1.310243e+00,1.293950e+00,1.283945e+00,1.333420e+00,1.314343e+00,1.179639e+00,1.281116e+00
40,3.443745e-01,4.751281e-01,4.705770e-01,4.411903e-01,3.910495e-01,4.440097e-01,4.732686e-01,4.570206e-01,4.551006e-01,4.619303e-01,4.253913e-01,4.496026e-01,4.541112e-01,3.618518e-01,4.410239e-01
60,1.624913e-02,5.780194e-02,5.965800e-02,4.380488e-02,3.025256e-02,4.544644e-02,5.811900e-02,5.401394e-02,4.941600e-02,5.616938e-02,4.000313e-02,5.085769e-02,5.131694e-02,2.498119e-02,5.105106e-02
80,5.684342e-14,5.684342e-14,5.684342e-14,5.684342e-14,5.684342e-14,5.684342e-14,5.684342e-14,5.684342e-14,5.684342e-14,5.684342e-14,5.684342e-14,5.684342e-14,5.684342e-14,5.684342e-14,5.684342e-14


1.1239901686659999